##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 重みクラスタリングの総合ガイド

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> TensorFlow.orgで表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示{</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/model_optimization/guide/clustering/clustering_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード/a0}</a></td>
</table>

TensorFlow Model Optimization ツールキットの一部である*重みクラスタリング*の総合ガイドへようこそ。

このページでは、さまざまなユースケースを示し、それぞれで API を使用する方法を説明します。どの API が必要であるかを特定したら、[API ドキュメント](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/clustering)でパラメータと詳細を確認してください。

- 重みクラスタリングのメリットとサポート対象を確認する場合は、[概要](https://www.tensorflow.org/model_optimization/guide/clustering)をご覧ください。
- 単一のエンドツーエンドの例については、[重みクラスタリングの例](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_example)をご覧ください。

このガイドでは、次のユースケースについて説明しています。

- クラスタモデルを定義する
- クラスタモデルのチェックポイントと逆シリアル化
- クラスタモデルの精度を改善する
- デプロイのみについて、ステップを実行して圧縮のメリットを確認する必要があります。


## セットアップ


In [ ]:
! pip install -q tensorflow-model-optimization

import tensorflow as tf
import numpy as np
import tempfile
import os
import tensorflow_model_optimization as tfmot

input_dim = 20
output_dim = 20
x_train = np.random.randn(1, input_dim).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randn(1), num_classes=output_dim)

def setup_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(input_dim, input_shape=[input_dim]),
      tf.keras.layers.Flatten()
  ])
  return model

def train_model(model):
  model.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
  )
  model.summary()
  model.fit(x_train, y_train)
  return model

def save_model_weights(model):
  _, pretrained_weights = tempfile.mkstemp('.h5')
  model.save_weights(pretrained_weights)
  return pretrained_weights

def setup_pretrained_weights():
  model= setup_model()
  model = train_model(model)
  pretrained_weights = save_model_weights(model)
  return pretrained_weights

def setup_pretrained_model():
  model = setup_model()
  pretrained_weights = setup_pretrained_weights()
  model.load_weights(pretrained_weights)
  return model

def save_model_file(model):
  _, keras_file = tempfile.mkstemp('.h5') 
  model.save(keras_file, include_optimizer=False)
  return keras_file

def get_gzipped_model_size(model):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  keras_file = save_model_file(model)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)
  return os.path.getsize(zipped_file)

setup_model()
pretrained_weights = setup_pretrained_weights()

## クラスタモデルを定義する


### モデル全体のクラスタリング（Sequential と Functional）

モデルの精度を高めるための**ヒント**:

- この API には許容できる精度のトレーニング済みモデルを渡す必要があります。クラスタリングを使用してモデルを最初からトレーニングすると、精度が低くなります。
- 一部のケースでは、特定のレイヤーをクラスタリングすると、モデルの精度に悪影響が及びます。精度に最も大きく影響するレイヤーのクラスタリングを省略する方法について、「一部のレイヤーをクラスタリングする」をご覧ください。

すべてのレイヤーをクラスタリングするには、モデルに `tfmot.clustering.keras.cluster_weights` を適用します。


In [ ]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 3,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

model = setup_model()
model.load_weights(pretrained_weights)

clustered_model = cluster_weights(model, **clustering_params)

clustered_model.summary()

### 一部のレイヤーをクラスタリングする（Sequential モデルと Functional モデル）


モデルの精度を高めるための**ヒント**:

- You must pass a pre-trained model with acceptable accuracy to this API. Training models from scratch with clustering results in subpar accuracy.
- 初期のレイヤーと比較し、後のレイヤーはより多い冗長パラメータ（`tf.keras.layers.Dense`、`tf.keras.layers.Conv2D` など）でクラスタリングします。
- 微調整中、クラスタリングレイヤーの前に初期のレイヤーを凍結します。凍結したレイヤーの数をハイパーパラメータとして処理します。経験的に、現在のクラスタリング API では、最も初期のレイヤーを凍結することが理想的です。
- クリティカルレイヤー（注意メカニズムなど）のクラスリングを回避します。

**その他**: `tfmot.clustering.keras.cluster_weights` API ドキュメントには、レイヤーごとにクラスタ構成を変える方法が示されています。

In [ ]:
# Create a base model
base_model = setup_model()
base_model.load_weights(pretrained_weights)

# Helper function uses `cluster_weights` to make only 
# the Dense layers train with clustering
def apply_clustering_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return cluster_weights(layer, **clustering_params)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_clustering_to_dense` 
# to the layers of the model.
clustered_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_clustering_to_dense,
)

clustered_model.summary()

### カスタム Keras レイヤーをクラスター化またはクラスター化するレイヤーの重みを指定する

`tfmot.clustering.keras.ClusterableLayer` は 2 つのユースケースに役立ちます。

1. カスタム Keras レイヤーなど、ネイティブでサポートされていないレイヤーをクラスター化する。
2. サポートされているレイヤーの内、クラスター化する重みを指定する。

たとえば、API はデフォルトで、`Dense` レイヤーのカーネルのみをクラスター化します。以下の例では、バイアスもクラスター化するようにデフォルトを変更する方法を示します。Keras レイヤーから派生させる場合、`get_clusterable_weights` 関数をオーバーライドして、クラスター化するトレーニング対象変数の名前とトレーニング対象変数自体を指定する必要があることに注意してください。たとえば、空の [] リストを返す場合、重みをクラスター化することはできません。

**一般的な過ち:** バイアスをクラスター化すると、通常、モデルの精度を著しく悪化させてしまいます。

In [ ]:
class MyDenseLayer(tf.keras.layers.Dense, tfmot.clustering.keras.ClusterableLayer):

  def get_clusterable_weights(self):
   # Cluster kernel and bias. This is just an example, clustering
   # bias usually hurts model accuracy.
   return [('kernel', self.kernel), ('bias', self.bias)]

# Use `cluster_weights` to make the `MyDenseLayer` layer train with clustering as usual.
model_for_clustering = tf.keras.Sequential([
  tfmot.clustering.keras.cluster_weights(MyDenseLayer(20, input_shape=[input_dim]), **clustering_params),
  tf.keras.layers.Flatten()
])

model_for_clustering.summary()

また、`tfmot.clustering.keras.ClusterableLayer` を使用して、Keras カスタムレイヤーをクラスター化することも可能です。これを行うには、通常どおりに `tf.keras.Layer` を拡張して `__init__`、`call`、および `build` 関数を実装しますが、`clusterable_layer.ClusterableLayer` クラスも拡張して、以下のように実装する必要があります。

1. `get_clusterable_weights` - 上記に示すように、クラスター化される重みカーネルを指定します。
2. `get_clusterable_algorithm` - 重みテンソルのクラスター化アルゴリズムを指定します。これは、クラスタリング用にカスタムレイヤーの重みをどのように形成するかを指定する必要があるためです。返されるクラスタリングアルゴリズムクラスは `clustering_algorithm.ClusteringAlgorithm` クラスから派生し、関数 `get_pulling_indices` がオーバーライドされます。この関数の例は[こちら](https://github.com/tensorflow/model-optimization/blob/18e87d262e536c9a742aef700880e71b47a7f768/tensorflow_model_optimization/python/core/clustering/keras/clustering_algorithm.py#L62)にあり、階数 1D、2D、および 3D の重みをサポートしています。

このユースケースの例は、[こちら](https://github.com/tensorflow/model-optimization/blob/master/tensorflow_model_optimization/python/core/clustering/keras/mnist_clusterable_layer_test.py)にあります。

## モデルのチェックポイントと逆シリアル化

**ユースケース:** このコードは、HDF5 モデル形式のみで必要です（HDF5 重みまたはその他の形式では不要です）。

In [ ]:
# Define the model.
base_model = setup_model()
base_model.load_weights(pretrained_weights)
clustered_model = cluster_weights(base_model, **clustering_params)

# Save or checkpoint the model.
_, keras_model_file = tempfile.mkstemp('.h5')
clustered_model.save(keras_model_file, include_optimizer=True)

# `cluster_scope` is needed for deserializing HDF5 models.
with tfmot.clustering.keras.cluster_scope():
  loaded_model = tf.keras.models.load_model(keras_model_file)

loaded_model.summary()

## クラスタモデルの精度を改善する

特定のユースケースについて、次のヒントを考慮できます。

- 最終的な最適化モデルの精度には、重心の初期化に重要な役割があります。一般的に、kmeans++ 初期化は、線形、密度、およびランダム初期化のパフォーマンスを上回ります。kmeans++ を使用しない場合、線形初期化は大きな重みを見逃さない傾向があるため、密度とランダム初期化のパフォーマンスを上回ります。ただし、密度初期化は、二峰性分布で重みに非常に少数のクラスターを使用した場合に、より優れた精度を示すことが観測されています。

- クラスタモデルを微調整する際は、トレーニングに使用されている学習率よりも低い率を設定します。

- モデルの精度を改善するための一般的なアイデアについては、「クラスタモデルを定義する」に記載のケース別のヒントをご覧ください。

## デプロイ

### サイズ圧縮によるモデルのエクスポート

**一般的な過ち**: `strip_clustering` と標準圧縮アルゴリズム（gzip など）の適用は、クラスタリングの圧縮のメリットを確認する上で必要です。

In [ ]:
model = setup_model()
clustered_model = cluster_weights(model, **clustering_params)

clustered_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

clustered_model.fit(
    x_train,
    y_train
)

final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

print("final model")
final_model.summary()

print("\n")
print("Size of gzipped clustered model without stripping: %.2f bytes" 
      % (get_gzipped_model_size(clustered_model)))
print("Size of gzipped clustered model with stripping: %.2f bytes" 
      % (get_gzipped_model_size(final_model)))